In [ ]:
# Install Library
# !pip install tweepy
# !pip install pandas

In [ ]:
# Import Library
import tweepy as tw
import csv
import pandas as pd
from datetime import timedelta, datetime

# Inisialisasi Akses Twitter API

In [ ]:
access_token = '1215248323579236353-c1VL0Ti9bntvKPHNdX4vWjKvzxmrvB'
access_token_secret = 'NachpxfacgvB1uQ5zfKli8seAvYkDHi0ULjqqXYEGSUb2'
consumer_key = '1jyHN0gmb7kOrPs7raCZuYwNZ'
consumer_key_secret = 'nrFnPcbVUAS7N1tHQowC08DdtHakyLr3vR1cEEhMRsbbwG02R7'

# Autentikasi Ke Twitter API

In [ ]:
auth = tw.OAuthHandler(consumer_key,consumer_key_secret)
auth.set_access_token(access_token,access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Inisialiasisi Fungsi Scrape Tweet

In [ ]:
def scrape_tweets_by_keyword(keyword,total=2000):
    tweets_list = []
    tweets = api.search_tweets(q=keyword,lang="id",count=total,tweet_mode="extended")
    
    for tweet in tweets:
        _tweet = {
            "id" : tweet.id_str,
            "username" : tweet.user.screen_name,
            "text" : tweet.full_text,
            "retweet_count" : tweet.retweet_count,
            "like_count" : tweet.favorite_count,
            "created_at" : tweet.created_at,
            "in_reply_to_status_id_str" : tweet.in_reply_to_status_id_str
        }
        tweets_list.append(_tweet)
    df_tweets = pd.DataFrame(tweets_list)
    return df_tweets

def scrape_tweets_by_username(username,total):
    tweets_list = []
    tweets = api.user_timeline(screen_name=username,count=total)
    
    for tweet in tweets:
        _tweet = {
            "id" : tweet.id_str,
            "username" : tweet.user.screen_name,
            "text" : tweet.text,
            "retweet_count" : tweet.retweet_count,
            "like_count" : tweet.favorite_count,
            "created_at" : tweet.created_at,
        }
        tweet_replies = scrape_tweets_by_keyword("to:"+tweet.user.screen_name)
        tweet_reply_in_id = []
        for index, tweet_reply in tweet_replies.iterrows():
            if tweet_reply['in_reply_to_status_id_str']==tweet.id_str:
                tweet_reply_in_id.append(tweet_reply.to_dict())
        _tweet['tweet_replies'] = tweet_reply_in_id
        
        tweets_list.append(_tweet)
        
        df_tweets = pd.DataFrame(tweets_list)
    return df_tweets

# Scrape Berdasarkan Username

In [ ]:
df_tweets_by_username = scrape_tweets_by_username("@RizkyBillar",1000)
print("Proses Scrape Tweet By Username Selesai. Total ",df_tweets_by_username.count())

Proses Scrape Tweet By Username Selesai. Total  id               4
username         4
text             4
retweet_count    4
like_count       4
created_at       4
dtype: int64


# Scrape Berdasarkan Keyword

In [ ]:
today = datetime.today().strftime("%Y-%m-%d")
last_week = (datetime.today() - timedelta(30)).strftime("%Y-%m-%d") #30 hari terakhir
date_since = last_week
date_until = today
df_tweets_by_keyword =  scrape_tweets_by_keyword("Rizky Billar",1000)
print("Proses Scrape Tweet By Keyword Selesai. Total ",df_tweets_by_keyword.count())

## Simpan Hasil Scraping

In [ ]:
filename = "Hasil_Scrape_Tweet_By_Username.csv"
df_tweets_by_username.to_csv(filename)
filename = "Hasil_Scrape_Tweet_By_Keyword.csv"
df_tweets_by_keyword.to_csv(filename)